In [ ]:
import DeepFMKit.core as dfm
import numpy as np
from tqdm import tqdm
import multiprocessing
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
import pickle

from DeepFMKit.workers import compare_fitter_stability

def generate_fitter_comparison_data(
    m_range=np.linspace(2, 25, 32),
    distortion_range=np.linspace(0, 0.2, 32),
    n_phase_trials=50,
    m_witness=0.1,
    n_cores=None
):
    """
    Computes the data for a 4-panel landscape plot comparing the stability
    of the simultaneous NLS and the sequential bootstrap W-DFMI fitters.
    """
    if n_cores is None:
        n_cores = os.cpu_count()
        
    print("=" * 60)
    print("Generating Fitter Comparison Landscape Data")
    print(f"Grid size: {len(m_range)} (m) x {len(distortion_range)} (eps)")
    print("=" * 60)
    
    jobs = []
    for j, m_main in enumerate(m_range):
        for i, eps in enumerate(distortion_range):
            jobs.append({
                'm_main': m_main, 'epsilon': eps, 'n_phase_trials': n_phase_trials,
                'm_witness': m_witness, 'grid_i': i, 'grid_j': j
            })

    grid_shape = (len(distortion_range), len(m_range))
    mean_unstable_grid = np.zeros(grid_shape)
    worst_unstable_grid = np.zeros(grid_shape)
    mean_sequential_grid = np.zeros(grid_shape)
    worst_sequential_grid = np.zeros(grid_shape)

    if __name__ == "__main__":
        with multiprocessing.Pool(processes=n_cores) as pool:
            results_iterator = pool.imap(compare_fitter_stability, jobs)
            for result in tqdm(results_iterator, total=len(jobs), desc="Calculating Grid Points"):
                i, j, mean_unstable, worst_unstable, mean_sequential, worst_sequential = result
                mean_unstable_grid[i, j] = mean_unstable
                worst_unstable_grid[i, j] = worst_unstable
                mean_sequential_grid[i, j] = mean_sequential
                worst_sequential_grid[i, j] = worst_sequential
    
    results = {
        "m_range": m_range,
        "distortion_range": distortion_range,
        "mean_unstable_grid": mean_unstable_grid,
        "worst_unstable_grid": worst_unstable_grid,
        "mean_sequential_grid": mean_sequential_grid,
        "worst_sequential_grid": worst_sequential_grid
    }
    
    return results

# --- Run the computation ---
if __name__ == "__main__":
    comparison_results = generate_fitter_comparison_data()
    
    # Save the results to disk so you don't have to run this again
    with open('fitter_comparison_results.pkl', 'wb') as f:
        pickle.dump(comparison_results, f)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap

def plot_fitter_comparison_landscape(results):
    """
    Plots the definitive 4-panel landscape figure comparing fitter stability.
    """
    m_range = results["m_range"]
    distortion_range = results["distortion_range"]
    mean_unstable_grid = results["mean_unstable_grid"]
    worst_unstable_grid = results["worst_unstable_grid"]
    mean_sequential_grid = results["mean_sequential_grid"]
    worst_sequential_grid = results["worst_sequential_grid"]
    
    fig = plt.figure(figsize=(18, 14))
    gs = gridspec.GridSpec(2, 3, width_ratios=[1, 1, 0.05], wspace=0.3, hspace=0.4)
    axes = [fig.add_subplot(gs[i, j]) for i in range(2) for j in range(2)]
    cax = fig.add_subplot(gs[:, 2])
    
    colors = ["#FFFFFF", "#D6EAF8", "#5499C7", "#154360"]
    custom_cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)
    vmax = np.max(np.abs(worst_unstable_grid))

    data_to_plot = [
        (mean_unstable_grid, '(a) Simultaneous NLS: Mean Bias'),
        (mean_sequential_grid, '(b) Sequential Fitter: Mean Bias'),
        (worst_unstable_grid, '(c) Simultaneous NLS: Worst-Case Bias'),
        (worst_sequential_grid, '(d) Sequential Fitter: Worst-Case Bias (Corrected)')
    ]

    for i, (ax, (data, title)) in enumerate(zip(axes, data_to_plot)):
        im = ax.pcolormesh(m_range, distortion_range * 100, np.abs(data),
                           cmap=custom_cmap, vmin=0, vmax=3, shading='auto')
        ax.set_title(title, fontsize=16, pad=10)
        ax.set_ylabel(r'2nd Harmonic Distortion, $\epsilon$ (%)', fontsize=14)
        if i >= 2: # Only add x-label to bottom row
            ax.set_xlabel(r'Modulation Depth, $m$', fontsize=14)
        ax.grid(False)

    fig.colorbar(im, cax=cax, orientation='vertical', label=r'Absolute Bias, $|\delta m|$ (rad)')
    
    plt.show()

# --- Load and plot the results ---
with open('fitter_comparison_results.pkl', 'rb') as f:
    comparison_results = pickle.load(f)

plot_fitter_comparison_landscape(comparison_results)

In [ ]:
import DeepFMKit.core as dfm
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import numpy as np
from tqdm import tqdm
import multiprocessing
import os
import pickle

from DeepFMKit.workers import calculate_bias_for_m_vs_mwitness

def generate_witness_interaction_data(
    m_main_range=np.linspace(3, 25, 50), # Reduced grid for faster testing
    m_witness_range=np.linspace(0.1, 1.0, 50),
    fitter_name='fit_wdfmi_orthogonal_demodulation',
    n_cores=None
):
    """
    Computes the W-DFMI bias landscape using the corrected physical model.
    This version is safe to run from a Jupyter Notebook.
    """
    if n_cores is None:
        n_cores = os.cpu_count()
        
    print("=" * 60)
    print("Generating Witness Interaction Bias Landscape (Corrected Physics)")
    print(f"Fitter: {fitter_name}")
    print(f"Grid size: {len(m_main_range)} (m_main) x {len(m_witness_range)} (m_witness)")
    print("=" * 60)
    
    jobs = []
    for j, m_main in enumerate(m_main_range):
        for i, m_witness in enumerate(m_witness_range):
            jobs.append({
                'm_main': m_main,
                'm_witness': m_witness,
                'fitter_func_name': fitter_name,
                'grid_i': i, 'grid_j': j
            })

    grid_shape = (len(m_witness_range), len(m_main_range))
    bias_grid = np.zeros(grid_shape)

    # This check is good practice for notebook environments
    if __name__ == "__main__":
        with multiprocessing.Pool(processes=n_cores) as pool:
            # Use imap for progress bar
            results_iterator = pool.imap(calculate_bias_for_m_vs_mwitness, jobs)
            for result in tqdm(results_iterator, total=len(jobs), desc="Calculating Grid Points"):
                i, j, bias = result
                bias_grid[i, j] = bias
            
    results = {
        "m_main_range": m_main_range,
        "m_witness_range": m_witness_range,
        "bias_grid": bias_grid,
        "fitter_name": fitter_name
    }
    
    return results

In [ ]:
# --- Main execution block ---
if __name__ == "__main__":
    # You can now easily switch between fitters here
    # Fitter choice 1: The robust Variable Projection method
    fitter_to_test = 'fit_wdfmi_orthogonal_demodulation'

    # Fitter choice 2: The sequential bootstrap method
    # fitter_to_test = 'fit_wdfmi_sequential'
    
    results_filename = f'interaction_results_{fitter_to_test}.pkl'
    
    # Run the simulation
    interaction_results = generate_witness_interaction_data(fitter_name=fitter_to_test)
    
    # Save the results
    with open(results_filename, 'wb') as f:
        pickle.dump(interaction_results, f)

In [ ]:
# Load and plot
with open(results_filename, 'rb') as f:
    loaded_results = pickle.load(f)

In [ ]:
def plot_witness_interaction_landscape(results):
    """
    Plots the W-DFMI bias landscape.
    """
    m_main_range = results["m_main_range"]
    m_witness_range = results["m_witness_range"]
    bias_grid = results["bias_grid"]
    fitter_name = results["fitter_name"]
    
    fig, ax = plt.subplots(figsize=(12, 9))

    # We expect bias to be very small now, so we can use a more sensitive color scale
    # but cap it at pi to see any remaining catastrophic failures.
    max_abs_bias = np.pi
    norm = Normalize(vmin=0, vmax=max_abs_bias)

    im = ax.pcolormesh(
        m_main_range, m_witness_range, np.abs(bias_grid),
        cmap='coolwarm', norm=norm, shading='auto'
    )
    
    ax.set_title(f'W-DFMI Residual Bias (Corrected Physics), Fitter: {fitter_name}', fontsize=16)
    ax.set_xlabel(r'Main Modulation Depth, $m_{\rm main}$', fontsize=14)
    ax.set_ylabel(r'Witness Modulation Depth, $m_{\rm witness}$', fontsize=14)
    
    cbar = fig.colorbar(im, ax=ax, label=r'Absolute Residual Bias, $|\delta m|$ (rad)')
    
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

plot_witness_interaction_landscape(loaded_results)

In [ ]:
import DeepFMKit.core as dfm
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import numpy as np
from tqdm import tqdm
import multiprocessing
import os
import pickle

# Import the new stochastic worker function
from DeepFMKit.workers import calculate_bias_for_m_vs_mwitness_stochastic

def generate_stochastic_bias_data(
    m_main_range=np.linspace(3, 25, 50),
    m_witness_range=np.linspace(0.1, 4.0, 50),
    fitter_name='fit_wdfmi_orthogonal_demodulation',
    n_trials_per_point=10,
    witness_phi_uncertainty_deg=10.0,
    n_cores=None
):
    """
    Computes the W-DFMI mean and worst-case bias landscapes under
    stochastic witness phase conditions.
    """
    if n_cores is None:
        n_cores = os.cpu_count()
        
    witness_phi_uncertainty_rad = np.deg2rad(witness_phi_uncertainty_deg)
    
    print("=" * 60)
    print("Generating Stochastic Witness Bias Landscape")
    print(f"Fitter: {fitter_name}")
    print(f"Grid size: {len(m_main_range)} x {len(m_witness_range)}")
    print(f"Trials per point: {n_trials_per_point}")
    print(f"Witness Phase Uncertainty: +/- {witness_phi_uncertainty_deg} deg")
    print("=" * 60)
    
    jobs = []
    for j, m_main in enumerate(m_main_range):
        for i, m_witness in enumerate(m_witness_range):
            jobs.append({
                'm_main': m_main,
                'm_witness': m_witness,
                'fitter_func_name': fitter_name,
                'n_trials': n_trials_per_point,
                'witness_phi_uncertainty_rad': witness_phi_uncertainty_rad,
                'grid_i': i, 'grid_j': j,
                'force_witness_phase': None
            })

    grid_shape = (len(m_witness_range), len(m_main_range))
    mean_bias_grid = np.zeros(grid_shape)
    worst_case_bias_grid = np.zeros(grid_shape)

    if __name__ == "__main__":
        with multiprocessing.Pool(processes=n_cores) as pool:
            results_iterator = pool.imap(calculate_bias_for_m_vs_mwitness_stochastic, jobs)
            for result in tqdm(results_iterator, total=len(jobs), desc="Calculating Stochastic Grid"):
                i, j, mean_bias, worst_case_bias = result
                mean_bias_grid[i, j] = mean_bias
                worst_case_bias_grid[i, j] = worst_case_bias
            
    results = {
        "m_main_range": m_main_range,
        "m_witness_range": m_witness_range,
        "mean_bias_grid": mean_bias_grid,
        "worst_case_bias_grid": worst_case_bias_grid,
        "fitter_name": fitter_name,
        "witness_phi_uncertainty_deg": witness_phi_uncertainty_deg
    }
    
    return results

In [ ]:
# --- Main execution block ---
if __name__ == "__main__":
    # --- Configuration ---
    fitter_to_test = 'fit_wdfmi_orthogonal_demodulation'
    uncertainty_deg = 15.0 # Test with a significant +/- 15 degree random error
    
    results_filename = f'stochastic_results_{fitter_to_test}_err{uncertainty_deg}deg.pkl'
    
    # --- Run Simulation ---
    stochastic_results = generate_stochastic_bias_data(
        fitter_name=fitter_to_test,
        witness_phi_uncertainty_deg=uncertainty_deg,
        n_trials_per_point=20 # Increase for smoother results, decrease for speed
    )
    
    # --- Save and Plot ---
    with open(results_filename, 'wb') as f:
        pickle.dump(stochastic_results, f)

In [ ]:
def plot_stochastic_bias_landscapes(results):
    """
    Plots the mean and worst-case W-DFMI bias landscapes in a two-panel figure.
    """
    m_main = results["m_main_range"]
    m_witness = results["m_witness_range"]
    mean_bias = results["mean_bias_grid"]
    worst_case_bias = results["worst_case_bias_grid"]
    fitter_name = results["fitter_name"]
    uncertainty = results["witness_phi_uncertainty_deg"]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7), sharey=True)
    fig.suptitle(f'W-DFMI Bias with Witness Phase Uncertainty of ±{uncertainty}°\nFitter: {fitter_name}', fontsize=18)

    # --- Panel 1: Mean Bias ---
    # Use a diverging colormap centered at zero for mean bias
    max_abs_mean = np.nanmax(np.abs(mean_bias))
    norm1 = Normalize(vmin=-max_abs_mean, vmax=max_abs_mean)
    im1 = ax1.pcolormesh(m_main, m_witness, mean_bias, cmap='coolwarm', norm=norm1, shading='auto')
    fig.colorbar(im1, ax=ax1, label=r'Mean Bias, $\langle \delta m \rangle$ (rad)')
    ax1.set_title('Mean Bias', fontsize=16)
    ax1.set_ylabel(r'Witness Modulation Depth, $m_{\rm witness}$', fontsize=14)
    ax1.grid(True, linestyle='--', alpha=0.6)

    # --- Panel 2: Worst-Case Bias ---
    # Use a sequential colormap for worst-case bias (always positive)
    norm2 = Normalize(vmin=0, vmax=np.pi) # Cap at pi to see catastrophic failures
    im2 = ax2.pcolormesh(m_main, m_witness, worst_case_bias, cmap='plasma', norm=norm2, shading='auto')
    fig.colorbar(im2, ax=ax2, label=r'Worst-Case Bias, max$|\delta m|$ (rad)')
    ax2.set_title('Worst-Case Bias', fontsize=16)
    ax2.grid(True, linestyle='--', alpha=0.6)

    # Common x-axis label
    for ax in [ax1, ax2]:
        ax.set_xlabel(r'Main Modulation Depth, $m_{\rm main}$', fontsize=14)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust for suptitle
    plt.show()

In [ ]:
with open(results_filename, 'rb') as f:
    loaded_results = pickle.load(f)
plot_stochastic_bias_landscapes(loaded_results)

In [ ]:
import DeepFMKit.core as dfm
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import numpy as np
from tqdm import tqdm
import multiprocessing
import os
import pickle

# Import the new worker function from your workers.py file
from DeepFMKit.workers import calculate_wdfmi_vs_dfmi_bias_with_distortion

def generate_distortion_bias_data(
    m_main_range=np.linspace(3, 25, 50),
    distortion_amp_range=np.linspace(0, 0.2, 50), # From 0% to 10% distortion
    m_witness_fixed=0.5,
    fitter_name='fit_wdfmi_orthogonal_demodulation',
    n_trials_per_point=20,
    n_cores=None
):
    """
    Computes the bias landscapes for W-DFMI vs conventional DFMI in the
    presence of 2nd harmonic frequency modulation distortion.
    """
    if n_cores is None:
        n_cores = os.cpu_count()
        
    print("=" * 60)
    print("Generating Distortion Bias Landscape (W-DFMI vs DFMI)")
    print(f"W-DFMI Fitter: {fitter_name}")
    print(f"Grid size: {len(m_main_range)} (m_main) x {len(distortion_amp_range)} (distortion)")
    print(f"Trials per point (random phase): {n_trials_per_point}")
    print("=" * 60)
    
    jobs = []
    for j, m_main in enumerate(m_main_range):
        for i, dist_amp in enumerate(distortion_amp_range):
            jobs.append({
                'm_main': m_main,
                'm_witness': m_witness_fixed,
                'distortion_amp': dist_amp,
                'fitter_func_name': fitter_name,
                'n_trials': n_trials_per_point,
                'grid_i': i, 'grid_j': j
            })

    grid_shape = (len(distortion_amp_range), len(m_main_range))
    wdfmi_mean_grid = np.zeros(grid_shape)
    wdfmi_worst_grid = np.zeros(grid_shape)
    dfmi_mean_grid = np.zeros(grid_shape)
    dfmi_worst_grid = np.zeros(grid_shape)

    if __name__ == "__main__":
        with multiprocessing.Pool(processes=n_cores) as pool:
            results_iterator = pool.imap(calculate_wdfmi_vs_dfmi_bias_with_distortion, jobs)
            for result in tqdm(results_iterator, total=len(jobs), desc="Calculating Distortion Grid"):
                i, j, w_mean, w_worst, d_mean, d_worst = result
                wdfmi_mean_grid[i, j] = w_mean
                wdfmi_worst_grid[i, j] = w_worst
                dfmi_mean_grid[i, j] = d_mean
                dfmi_worst_grid[i, j] = d_worst
            
    results = {
        "m_main_range": m_main_range,
        "distortion_amp_range": distortion_amp_range,
        "wdfmi_mean_grid": wdfmi_mean_grid,
        "wdfmi_worst_grid": wdfmi_worst_grid,
        "dfmi_mean_grid": dfmi_mean_grid,
        "dfmi_worst_grid": dfmi_worst_grid,
        "fitter_name": fitter_name
    }
    
    return results

In [ ]:
# --- Main execution block ---
if __name__ == "__main__":
    # --- Configuration ---
    fitter_to_test = 'fit_wdfmi_orthogonal_demodulation'
    results_filename = f'distortion_comparison_{fitter_to_test}.pkl'
    
    # --- Run Simulation ---
    distortion_results = generate_distortion_bias_data(
        fitter_name=fitter_to_test,
        n_trials_per_point=40
    )
    
    # --- Save and Plot ---
    with open(results_filename, 'wb') as f:
        pickle.dump(distortion_results, f)

In [ ]:

def plot_distortion_comparison(results):
    """
    Plots the four-panel comparison of W-DFMI vs conventional DFMI bias.
    """
    m_main = results["m_main_range"]
    dist_amp = results["distortion_amp_range"]
    fitter_name = results["fitter_name"]
    
    grids = {
        'W-DFMI Mean Bias': results["wdfmi_mean_grid"],
        'DFMI Mean Bias': results["dfmi_mean_grid"],
        'W-DFMI Worst-Case Bias': results["wdfmi_worst_grid"],
        'DFMI Worst-Case Bias': results["dfmi_worst_grid"]
    }

    fig, axes = plt.subplots(2, 2, figsize=(16, 12), sharex=True, sharey=True)
    axes = axes.flatten()
    fig.suptitle(f'Algorithm Performance with 2nd Harmonic Distortion\nW-DFMI Fitter: {fitter_name}', fontsize=18)

    # Use a common color scale for worst-case bias to allow direct comparison
    max_worst_bias = np.nanmax([grids['W-DFMI Worst-Case Bias'], grids['DFMI Worst-Case Bias']])
    max_worst_bias = min(max_worst_bias, np.pi) # Cap at pi
    norm_worst = Normalize(vmin=0, vmax=max_worst_bias)
    
    # Use a common color scale for mean bias
    max_mean_bias = np.nanmax(np.abs([grids['W-DFMI Mean Bias'], grids['DFMI Mean Bias']]))
    norm_mean = Normalize(vmin=-np.pi, vmax=np.pi)

    plot_params = {
        'W-DFMI Mean Bias': {'cmap': 'coolwarm', 'norm': norm_mean, 'label': r'$\langle \delta m \rangle$ (rad)'},
        'DFMI Mean Bias': {'cmap': 'coolwarm', 'norm': norm_mean, 'label': r'$\langle \delta m \rangle$ (rad)'},
        'W-DFMI Worst-Case Bias': {'cmap': 'viridis', 'norm': norm_worst, 'label': r'max$|\delta m|$ (rad)'},
        'DFMI Worst-Case Bias': {'cmap': 'viridis', 'norm': norm_worst, 'label': r'max$|\delta m|$ (rad)'}
    }

    for i, (title, grid_data) in enumerate(grids.items()):
        ax = axes[i]
        params = plot_params[title]
        im = ax.pcolormesh(m_main, dist_amp, grid_data, cmap=params['cmap'], norm=params['norm'], shading='auto')
        fig.colorbar(im, ax=ax, label=params['label'])
        ax.set_title(title, fontsize=16)
        ax.grid(True, linestyle='--', alpha=0.6)

    # Set common labels
    for ax in axes:
        ax.set_xlabel(r'Main Modulation Depth, $m_{\rm main}$', fontsize=14)
        ax.set_ylabel(r'Distortion Amplitude, $\epsilon$', fontsize=14)
    
    # Hide redundant labels
    for ax in [axes[0], axes[2]]:
        ax.set_xlabel('')
    for ax in [axes[1], axes[3]]:
        ax.set_ylabel('')

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

In [ ]:
with open(results_filename, 'rb') as f:
    loaded_results = pickle.load(f)
plot_distortion_comparison(loaded_results)